In [1]:
import os
os.chdir(r"C:\Users\cornuch\Desktop\SISE\Fouille_donnees_massives\projet_fouille")  

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [3]:
df = pd.read_table("guillaume.txt", sep=";", header=0, decimal=",")
df=df.drop(df[df["CodeDecision"]==4].index, axis= 0)

# CodeDecision et FlagImpaye

In [5]:
df['CodeDecision'].value_counts(normalize=True)

0    0.753811
1    0.241377
2    0.004210
3    0.000602
Name: CodeDecision, dtype: float64

Les clients "douteux" correspondent à environ 0.48 % de l'ensemble de la clientèle

In [6]:
df_douteux=df.iloc[np.where(df['CodeDecision']>=2)[0], :]
df_non_douteux=df.iloc[np.where(df['CodeDecision']<2)[0], :]

In [7]:
percent_d=df_douteux['FlagImpaye'].value_counts(normalize=True)*100
percent_nd=df_non_douteux['FlagImpaye'].value_counts(normalize=True)*100

In [8]:
data_barplot = pd.DataFrame({ "Flag Impaye": [0,1],"client non douteux": [percent_nd[0],percent_d[0]],"client douteux":[percent_nd[1],percent_d[1]]})
data_barplot

,Flag Impaye,client non douteux,client douteux
0,0,99.779453,0.220547
1,1,11.466905,88.533095


fig = px.bar(data_barplot, x="Flag Impaye", y=["client non douteux", "client douteux"], title="Acceptation des transactions selon la typologie de clientèle")
fig.show()

Les deux variables semblent fortement liées. En effet, parmi les FlagImpaye, la proportion de clients douteux (CodeDecison supérieur ou égal à 2) est de 88,53% contre 0,22% pour les clients non douteux (CodeDecision inférieur à 2). 

In [5]:
from scipy.stats import chi2_contingency
df_count = pd.crosstab(df['CodeDecision'], df['FlagImpaye'])
chi2_contingency(df_count)

(2857837.2422457836,
 0.0,
 3,
 array([[3.48017545e+06, 2.26105490e+04],
        [1.11438588e+06, 7.24011677e+03],
        [1.94357269e+04, 1.26273075e+02],
        [2.77993886e+03, 1.80611422e+01]]))

On va calculer le V de Cramer car lorsque les effectifs sont très élevés, le test du χ2 aboutit quasi-systématiquement au rejet de l'hypothèse d'indépendance.

In [9]:
V=np.sqrt(chi2_contingency(df_count)[0]/df.shape[0])
V

0.7842292994424459

Le V de Cramer et le T de Tschuprow sont identiques dans notre cas. Celui-ci est élevé et renforce l'idée de dépendance entre ces variables.

# Montant et FlagImpaye

In [12]:
df_select_montant=df[df.Montant < 400]

In [13]:
df_select_montant.shape[0]*100/df.shape[0]

99.74151948922822

fig = px.box(df_select_montant, x='FlagImpaye', y='Montant',color='FlagImpaye')
fig.show()

In [20]:
m0=df[df.FlagImpaye==0]['Montant'].tolist()
m1=df[df.FlagImpaye==1]['Montant'].tolist()

from scipy.stats import ttest_ind
print(ttest_ind(m0,m1))
print(ttest_ind(m0,m1,equal_var=False))

In [29]:
from scipy.stats import mannwhitneyu
mannwhitneyu(m0, m1)

MannwhitneyuResult(statistic=52545555212.5, pvalue=0.0)

Rejet de l'indépendance entre les variables au risque de 5%
On pourrait faire de même avec les autres variables quantitatives